# Correction TP1

In [1]:
from A_star import main

## Algorithme A* sans heuristique
### La classe graphe
La classe graphe est déjà fournie avec le TP, elle lit un fichier texte et l'interprête comme un graphe. Le graphe est représenté implicitement en utilisant une matrice de coût. Une liste d'arête est cependant gardée pour connaitre leur ordre et trouver plus rapidement un plus court chemin.

Elle contient deux fonctions : une fonction qui renvoye une arête à partir de ses extrémités et une fonciton qui renvoye la liste triée des arêtes. La première fonction pourrait être remplacée par une fonction qui renvoie uniquement le coût de l'arête, puisque seule cette information est rajoutée. 
```python
def get_cost(self,i,j):
    return self.costs[i,j]
```
### La classe Solution
Il fallait tout d'abord implémenter le constructeur de la classe. Ici une dijonction de cas etait proposée par test de type. Il fallait faire attention ici à bien copier la liste des villes visitées et non visitées. Il ne fallait cependant pas copier le graphe sous peine de perdre beaucoup de temps de calcul. En effet le graphe étant le même pour toutes les solutions, et n'étant pas modifié au cours de l'algorithme, une simple référence suffit. 
```python
    def __init__(self, s=None):
        if isinstance(s, Graph):
            self.g = s
            self.cost = 0
            self.visited = []
            self.not_visited = list(range(s.N))
        elif isinstance(s, Solution):
            self.g = s.g
            self.cost = s.cost
            self.visited = copy.deepcopy(s.visited)
            self.not_visited = copy.deepcopy(s.not_visited)
        else:
            raise ValueError('you should give a graph or a solution')
```

Il fallait ensuite compléter la fonction qui rajoute une arête à la solution. Il n'y avait pas de difficutés particulière ici
```python
def add_edge(self, v, u):
    self.cost += self.g.get_edge(v, u).cost
    self.visited.append(u)
    self.not_visited.remove(u)
```
La dernière fonction devait afficher la solution, plusieur affichages sont corrects, on peut par exemple prendre la fonction suivante :
```python
def print(self):
    print('['+ str(0), end='')
    for i in self.visited:
        print(', '+str(i), end='')
    print(']')
    print('cost: ' + str(self.cost))
```
Pour ceux qui utilisent python 2.7 il fallait renommer cette fonction

## La classe Node
Cette classe implemente les noeuds de l'arbre de recherche. Chaque Noeud contient une solution ainsi qu'un coût actuel et garde en mémoire la dernière ville visitée. Il fallait d'abord l'initialiser :
```python
def __init__(self, v, sol, heuristic_cost=0):
    self.v = v
    self.solution = sol
    self.heuristic_cost = heuristic_cost
```
Pas de difficulté particulière ici. Il ne faut juste pas oublier d'initialiser aussi le cout de l'heuristique afin de ne l'oublier par la suite. Il est d'ailleur par défaut égal à 0. 

On pouvait ensuite créer une fonction `cost` qui renvoie le cout du neoud : égal au coût heuristique + le coût de la solution
```python
def cost(self):
    return self.solution.cost + self.heuristic_cost
```
Il était demandé ensuite d'implémenter la fonction `isN2betterThanN1(N1,N2)` qui compare deux noeuds. Une implémentation simple est la suivante : 
```python
def isN2betterThanN1(N1, N2):
    f1 = N1.solution.cost + N1.heuristic_cost
    f2 = N2.solution.cost + N2.heuristic_cost
    return f2 < f1
```
Une analyse plus poussée montre que on peut compléter la fonction en classant les noeud qui ont même coût par nombre de villes visitées décroissant. Cela donne:
```python
def isN2betterThanN1(N1, N2):
    f1 = N1.solution.cost + N1.heuristic_cost
    f2 = N2.solution.cost + N2.heuristic_cost
    if (f2 < f1):
        return True
    elif f2 == f1:
        if len(N2.solution.visited) < len(N1.solution.visited):
            return True
        elif (len(N2.solution.visited) == len(N1.solution.visited)) & (N2.solution.cost < N1.solution.cost):
            return True
        else:
            return False
    else:
        return False
```
En python il fallait aussi surcharger l'opérateur *<* pour que la file de priorité puisse marcher:
```python
def __lt__(self, other):
    return self.cost() < other.cost()
```
La fonction plus compliquée de cette partie était la fonction `explore_node(self, heap)` qui devait insérer dans `heap` tous les noeuds fils du noeud en cours. Il fallait faire attention à bien mettre à jour le noeud avant de l'insérer dans la file de priorité, qui suppose que ses éléments restent inchangés. Il fallait aussi s'assurer que le noeud retournant à la source n'etait créé qu'en dernier. Cela donne le code suivant:
```python
def explore_node(self, heap):
    if len(self.solution.not_visited) == 1:
        s = Solution(self.solution)
        s.add_edge(self.v, SOURCE)
        n = Node(SOURCE, s, heuristic_cost=0)
        heap.put(n)
        return
    for u in self.solution.not_visited:
        if u != SOURCE:
            s = Solution(self.solution)
            s.add_edge(self.v, u)
            n = Node(u, s, heuristic_cost=0)
            heap.put(n)
```
Il faudra modifier l'initialisation des noeuds qui ne terminent pas la solution pour utiliser l'heuristique. 

## Algorithme A*
Il faut maintenant implémenter la fonction principale de l'algorithme A\*. Cette fonction construit et explore la file de priorité tandis que `explore_node` la remplit. La première solution trouvée, par l'invariant de l'algorithme est la solution optimale. On rapelle que l'invariant de l'algorithme est que la solution partielle de cout minimal est toujours celle explorée. 
```python
def main(graph):
    g = Graph(graph)
    root = Node(SOURCE, Solution(g))
    heap = PriorityQueue()
    root.explore_node(heap)
    k = 0
    while True:
        node = heap.get()
        k += 1
        if node.v == SOURCE:
            print('visited nodes ', k)
            print('created nodes ', k+heap.qsize())
            node.solution.print()
            return
        node.explore_node(heap)```
### Execution

In [2]:
main('N10.data','zero')

running time 4.56 s
visited nodes  41621
created nodes  116619
[0, 4, 6, 3, 5, 2, 9, 1, 8, 7, 0]
cost: 135.0


In [3]:
main('N12.data','zero')

running time 176.30 s
visited nodes  1164095
created nodes  4134560
[0, 9, 2, 4, 10, 6, 8, 1, 3, 11, 7, 5, 0]
cost: 1733.0


Plusieurs solutions sont réalisables, mais vous devriez tomber sur la solution ci dessus dans ce sens ou dans l'autre. Cet algorithme n'est pas assez efficace pour résoudre le cas N15
## Heuristiques et MST
L'introduction d'une heuristique permet d'améliorer grandement la vitesse de l'algorithme. Le rôle de l'heuristique est d'ecarter les branches les moins prometteuses, plus specifiquement, de prioriser celle qui le sont plus. Le respect de la certaines conditions d'admissibilité permet de garantir de trouver la meilleure solution. Si cette condition n'est plus respectée, alors il peut exister des cas l'algorithme ne donne pas la meilleure solution. **Le fait que l'algorithme donne la bonne solution sur un sous ensemble de cas ne constitue en aucun cas la preuve de la justesse de l'algorithme**. Seule des garanties mathématiques ou la justesse de l'algorithme sur tous les cas peuvent en constituer une preuve. 

L'heuristique utilisée cherche (dans cette application) à estimer le coût restant du parcours. Pour être admissible une telle heuristique doit toujours sous estimer cette distance. La perte de cette condition entraine la perte de la justesse de l'algorithme. 

Nous allons utiliser pour estimer cette distance le MST. Cette approche est particulièrement intéressant puisque l'on que, par l'inégalité triangulaire, le tour optimal a une longueur comprise entre la longueur du MST et 2\* la longueur du MST. Ainsi le coût du MST constitue une bonne approximation au sous tour minimal. L'astuce ici consiste à calculer la l'arbre couvrant minimal qui contient la solution courante : cet arbre minimise la taille de tout tour contenant la solution courante. Pour le montrer il suffit de constater que si $S$ est une solution partielle, $A$ l'arbre convrant minimal contenant $S$ et $T$ le tour minimal contenant $S$. Alors si on enlève à $T$ n'importe quelle arête $e$ qui n'est pas dans $S$ alors $T-\{e\}$ est un arbre couvrant et par définition, son poids est supérieur ou égal à celui de $A$. Ainsi $Coût(A)<Coût(T)$ 

### Mise en place de l'algorithme
Il faut commencer par calculer le MST contenant la solution actuelle, pour cela il faut :
1. remettre à zéro l'*UnionFind*
2. parcourrir toutes les arêtes de la solution et les ajouter à l'*UnionFind*
3. lancer l'algorithme de Kruskal sur les arêtes restantes

L'Union find est un algorithme qui permet de trouver rapidement si deux sommet appartiennent à la même composante connexe. Il est initialisé en mettant chaque sommet dans un cluster. A chaque fois qu'une arête est insérée dans la structure, il fusionne les deux clusters des deux sommets de l'arête. Chaque cluster correspond ainsi à une composante connexe de la forêt d'arbres. Pour tester si deux sommets sont dans la même composante connexe il suffit de regarder à quel cluster ils appartiennent. 
Cette structure est particulièrement adaptée à l'algorithme de Kruskal, car il permet de compléter l'arbre en testant pour chaque arête si elle fait un cycle (sommets dans la même composante connexe). 
L'algorithme de Kruskal se présente donc ainsi :
```python
def getMSTCost(self, sol, source):
    mst_cost = 0
    self.uf.reset()
    u = source
    n_edges = 0
    for e in sol.visited:
        self.uf.add(self.g.get_edge(e, u))
        u = e
        n_edges += 1

    for e in self.g.get_sorted_edges():
        if n_edges == self.g.N - 1:
            return mst_cost
        elif not self.uf.makes_cycle(e):
            mst_cost += e.cost
            self.uf.add(e)
            n_edges += 1

    return mst_cost
```
Quelques points délicats : 
- ne pas oublier de remettre à zero l'*UnionFind*
- ne pas oublier la source
- pour la condition de fin, il suffit de compter le nombre d'arêtes insérée, il y en aura toujours N-1, comme dans tout arbre couvrant

Une fois Kruskal implémenté il faut l'intégrer à l'algorithme A\*. Pour cela on va calculer l'heuristique à chaque creation de Noeud. Il faut ainsi l'insérer soit dans le constructeur soit dans la fonction `explore_node`. Attention à la calculer avant l'insertion dans la file de priorité. Il ne faut pas oublier non plus d'insérer la ligne suivante au début de la fonction main:
````python
Kruskal.kruskal = Kruskal.Kruskal(g)
```
### Execution

In [4]:
main('N10.data','mst')

running time 4.69 s
visited nodes  15749
created nodes  44321
[0, 7, 8, 1, 9, 2, 5, 3, 6, 4, 0]
cost: 135.0


In [5]:
main('N12.data','mst')

running time 37.06 s
visited nodes  111010
created nodes  372273
[0, 5, 7, 11, 3, 1, 8, 6, 10, 4, 2, 9, 0]
cost: 1733.0


In [6]:
main('N15.data','mst')

running time 16.19 s
visited nodes  27383
created nodes  140319
[0, 12, 1, 14, 8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0]
cost: 291.0


On retrouve bien les même solutions que pour le premier algorithme. On observe aussi une forte réduction du nombre de noeuds visité, même si pour le premier algorithme la différence de temps n'est pas significative. Elle est en revanche divisée par 4 pour le deuxième exemple. L'algorithme arrice aussi à résoudre le troisième exemple. La quatrième exemple n'est pas solvable en temps raisonnable (plus de 3h). 

## Amélioration de l'heuristique
Il est possible de constater que la construction actuelle du MST n'est pas optimale est que certains coût supplémentaires peuvent être ajoutées. En effet la solution cherchée contiendra au moins une arête de plus de le MST. Il est donc posssible de considérer le cout d'une arête supplémentaire dans l'heuristique. Il faut maintenant déterminer laquelle. Une solution simple serait de rajouter la plus petite arête, non contenue dans le MST, au coût.
On peut raffiner cette solution en ajoutant le cout de la plus petite arête non contenue dans le MST qui ne touche pas la solution partielle.
Si on considère la dernière ville $u$ ajoutée à la solution, et la ville $v$ la plus proche non visitée. Il y a alors deux cas possibles : soit le $u$ est une feuille de l'arbre, et pour compléter le tour il faudra ajouter une arête de poids au moins $c(e_{uv})$. Soit $u$ n'est pas une feuille, alors l'arête à ajouter pour completer le tour aura un coût au moin $c(e_{uv})$ par construction du MST. 
En allant plus loin on peut montrer qu'on peut ajouter le poids de l'arête suivante de la dernière arête ajoutée au MST. On ne peut ajouter une meilleure arête au MST sans perdre de garantie.
Dans cette construction du MST une seule arête peut être ajoutée. Un raisonnement similaire permet de montrer que l'on peut ajouter l'arête qui part de la source et qui va vers la ville non visitée la plus proche, mais on ne peut les ajouter en même temps.

**Remarque** : les arêtes qu'il est possible d'ajouter dépendent du MST choisi. 


L'ajout de la ville visité la plus proche de al source peut être implémenté de la façon suivante :

```python
def getMSTCost(self, sol, source):
    mst_cost = 0
    self.uf.reset()
    u = source
    n_edges = 0
    for e in sol.visited:
        self.uf.add(self.g.get_edge(e, u))
        u = e
        n_edges += 1
    for e in self.g.get_sorted_edges():
        if n_edges == self.g.N - 1:
            mst_cost += np.min(self.g.costs[0, sol.not_visited])
            return mst_cost
        elif not self.uf.makes_cycle(e):
            mst_cost += e.cost
            self.uf.add(e)
            n_edges += 1
    mst_cost += np.min(self.g.costs[0, sol.not_visited])
    return mst_cost
```
L'utilisation de la meilleure arête par :
```python
def getMSTCost(self, sol, source):
    mst_cost = 0
    self.uf.reset()
    u = source
    n_edges = 0
    for e in sol.visited:
        self.uf.add(self.g.get_edge(e, u))
        u = e
        n_edges += 1
    for e in self.g.get_sorted_edges():
        if n_edges == self.g.N - 1:
            mst_cost += e.cost
            return mst_cost
        elif not self.uf.makes_cycle(e):
            mst_cost += e.cost
            self.uf.add(e)
            n_edges += 1
    mst_cost += np.min(self.g.costs[0, sol.not_visited])
    return mst_cost
```

In [7]:
main('N10.data','mst_source')

running time 5.05 s
visited nodes  15749
created nodes  44321
[0, 7, 8, 1, 9, 2, 5, 3, 6, 4, 0]
cost: 135.0


In [8]:
main('N12.data','mst_source')

running time 44.57 s
visited nodes  111010
created nodes  372273
[0, 5, 7, 11, 3, 1, 8, 6, 10, 4, 2, 9, 0]
cost: 1733.0


In [9]:
main('N15.data','mst_source')

running time 18.28 s
visited nodes  27383
created nodes  140319
[0, 12, 1, 14, 8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0]
cost: 291.0


Malheuresement l'ajout de la ville non visitée la plus proche à la source ne permet pas d'améliorer de façon significative la preformance des algorithmes, le nombre d'états visité reste le même mais le temps de calcul augmente. 

In [10]:
main('N10.data','mst_best_edge')

running time 1.71 s
visited nodes  6092
created nodes  18279
[0, 4, 6, 3, 5, 2, 9, 1, 8, 7, 0]
cost: 135.0


In [11]:
main('N12.data','mst_best_edge')

running time 9.35 s
visited nodes  24445
created nodes  87744
[0, 9, 2, 4, 10, 6, 8, 1, 3, 11, 7, 5, 0]
cost: 1733.0


In [12]:
main('N15.data','mst_best_edge')

running time 0.20 s
visited nodes  310
created nodes  1706
[0, 12, 1, 14, 8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0]
cost: 291.0


On constate par contre que l'ajout de la meilleure arête a une influence significative sur le résultat. L'algorithme est beaucoup plus efficace divisant par plus de 4 le temps calcul. On optient toujours les bon résultat, ce qui est rassurant mais ne constitue pas une preuve non plus.  

## Bonus 
* Une implémentation du MST plus efficace construit le MST uniquement sur les villes non visitée et sur la source et la dernière ville visitée. Cette heuristique est particulièrement efficace et permet de traiter en moins d'une heure le test N17. 

Le code correspondant à cette methode est le suivant :
```python
def getMSTCost(self, sol, source):
    mst_cost = 0
    self.uf.reset()
    n_edges = 0
    if len(sol.visited)>0:
        for e in self.g.get_sorted_edges():
            if n_edges == self.g.N - len(sol.visited):
                return mst_cost
            elif not (self.uf.makes_cycle(e) | (e.source in sol.visited[:-1]) | (e.destination in sol.visited[:-1])):
                mst_cost += e.cost
                self.uf.add(e)
                n_edges += 1
    return mst_cost
```

In [13]:
main('N10.data','best_mst')

running time 0.99 s
visited nodes  2178
created nodes  7735
[0, 4, 9, 2, 5, 3, 6, 1, 8, 7, 0]
cost: 135.0


In [14]:
main('N12.data','best_mst')

running time 0.84 s
visited nodes  902
created nodes  5105
[0, 5, 7, 11, 3, 1, 8, 6, 10, 4, 2, 9, 0]
cost: 1733.0


In [15]:
main('N15.data','best_mst')

running time 0.09 s
visited nodes  33
created nodes  262
[0, 12, 1, 14, 8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0]
cost: 291.0


In [16]:
main('N17.data','best_mst')

running time 1684.82 s
visited nodes  659399
created nodes  5352457
[0, 3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 9, 1, 4, 8, 11, 15, 0]
cost: 2085.0


On voit donc que cette implémentation est encore plus efficace que l'amélioration précedante en divisant encore par deux le temps de calcul. Le nombre de noeuds visités devenant très faible.
* Il existe de nombreuses heuristiques qui approchent la solution optimale, la plus classique étant la solution de recherche locale à partir d'une solution quelconque. On peut aussi utiliser le MST pour construire une solution approchée 2-optimale. L'algorithme par colonie de fourmis permet aussi d'atteindre des bonnes solutions mais demande plus de puissance de calcul. 

### Résumé des performances des algorithmes

|         | zero |               |  MST  |               |             | MST source |               |             | MST best edge |               |             | MST Bonus |               |             |
|:-------:|:----:|:-------------:|:-----:|:-------------:|:-----------:|:----------:|:-------------:|:-----------:|:-------------:|:-------------:|:-----------:|:---------:|:-------------:|:-----------:|
| dataset | temps | noeuds visités |  temps | noeuds visités | amélioration |    temps    | noeuds visités | amélioration |      temps     | noeuds visités | amélioration |    temps   | noeuds visités | amélioration |
|   N10   | 5.16 |     41621     |  4.19 |     15749     |     18%     |    4.88    |     15749     |      5%     |      1.76     |      6092     |     65%     |    1.06   |      2178     |     79%     |
|   N12   |  176 |    1164095    | 37.37 |     111010    |     78%     |    45.15   |     111010    |     74%     |      9.01     |     24445     |     95%     |    0.92   |      902     |     99.4%     |
|   N15   |  NA  |       NA      | 16.59 |     27383     |      100%     |    18.47   |     27383     |     -11%    |      0.22     |      310      |    98.6%    |    0.04   |      33      |     99.7%     |
|   N17   |  NA  |       NA      |   NA  |       NA      |      NA     |     NA     |       NA      |      NA     |    NA           |    NA    |      NA     |    1684.82       |     659399          |     100%        |